# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:01<00:16,  1.40s/it]

 15%|█▌        | 2/13 [00:02<00:15,  1.37s/it]

 23%|██▎       | 3/13 [00:03<00:11,  1.11s/it]

 31%|███       | 4/13 [00:04<00:10,  1.17s/it]

...

 77%|███████▋  | 10/13 [00:11<00:03,  1.21s/it]

 85%|████████▍ | 11/13 [00:12<00:02,  1.25s/it]

 92%|█████████▏| 12/13 [00:14<00:01,  1.32s/it]

100%|██████████| 13/13 [00:15<00:00,  1.32s/it]

100%|██████████| 13/13 [00:15<00:00,  1.20s/it]

In [7]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,61860,"Bamsoet Ungkap, Dunia Internasional Kagumi Pan...",RM.id Rakyat Merdeka - Ketua MPR sekaligus Wa...,"Sabtu, 1 Juni 2024 16:01 WIB",,https://rm.id/,https://rm.id/baca-berita/parlemen/222647/upac...,2024-06-01 16:33:36.265070,17,True,UJANG SUNDA,UJANG SUNDA,False,2024-06-01 16:01:00,1,bamsoet ungkap dunia internasional kagumi panc...,"{bamsoet, kagumi, pancasila, internasional, du...",Neutral,0.644616
1,61859,"Di Depan BEM PTNU Se-Nusantara, Bahlil Dorong ...",RM.id Rakyat Merdeka - Menteri Investasi/Kepa...,"Sabtu, 1 Juni 2024 15:46 WIB",,https://rm.id/,https://rm.id/baca-berita/ekonomi-bisnis/22264...,2024-06-01 16:33:36.257516,17,False,DWI ILHAMIEditor :,SRI NURGANINGSIH,False,2024-06-01 15:46:00,1,di depan bem ptnu se nusantara bahlil dorong m...,"{dorong, mahasiswa, bem, nusantara, bahlil, pt...",Neutral,0.999673
2,61858,Jokowi Minta Pertamina Dan PLN Geber Energi Hi...,RM.id Rakyat Merdeka - Presiden Jokowi mengim...,"Sabtu, 1 Juni 2024 15:43 WIB",,https://rm.id/,https://rm.id/baca-berita/ekonomi-bisnis/22264...,2024-06-01 16:33:36.249971,17,False,FIRSTY HESTYARINI,FIRSTY HESTYARINI,False,2024-06-01 15:43:00,1,jokowi minta pertamina dan pln geber energi hi...,"{jokowi, alasannya, pln, hijau, geber, pertami...",Neutral,0.999761
3,61857,Update Haji: 146 Ribu Jemaah Telah Tiba di Tan...,RM.id Rakyat Merdeka - Berdasarkan laporan Pe...,"Sabtu, 1 Juni 2024 15:05 WIB",,https://rm.id/,https://rm.id/baca-berita/humaniora/222640/upd...,2024-06-01 16:33:36.243274,17,False,UJANG SUNDA,UJANG SUNDA,False,2024-06-01 15:05:00,1,update haji ribu jemaah telah tiba di tanah su...,"{tanah, suci, jemaah, haji, wafat, ribu, orang...",Neutral,0.999694
4,61856,"Perolehan Suara Melorot, Ketua PPP DKI Didesak...",RM.id Rakyat Merdeka - Kinerja pengurus Parta...,"Sabtu, 1 Juni 2024 15:04 WIB",,https://rm.id/,https://rm.id/baca-berita/parpol/222641/perole...,2024-06-01 16:33:36.236495,17,False,DEDE ISWADI IDRISEditor :,FAQIH MUBAROK,False,2024-06-01 15:04:00,1,perolehan suara melorot ketua ppp dki didesak ...,"{melorot, undur, suara, ppp, dki, ketua, dides...",Neutral,0.999639


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


In [9]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:02<00:29,  2.47s/it]

 15%|█▌        | 2/13 [00:05<00:28,  2.59s/it]

 23%|██▎       | 3/13 [00:07<00:26,  2.63s/it]

 31%|███       | 4/13 [00:10<00:22,  2.52s/it]

...

 77%|███████▋  | 10/13 [00:25<00:07,  2.51s/it]

 85%|████████▍ | 11/13 [00:29<00:05,  2.92s/it]

 92%|█████████▏| 12/13 [00:33<00:03,  3.23s/it]

100%|██████████| 13/13 [00:36<00:00,  3.26s/it]

100%|██████████| 13/13 [00:36<00:00,  2.81s/it]

In [10]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,61860,"Bamsoet Ungkap, Dunia Internasional Kagumi Pan...",RM.id Rakyat Merdeka - Ketua MPR sekaligus Wa...,"Sabtu, 1 Juni 2024 16:01 WIB",,https://rm.id/,https://rm.id/baca-berita/parlemen/222647/upac...,2024-06-01 16:33:36.265070,17,True,UJANG SUNDA,UJANG SUNDA,False,2024-06-01 16:01:00,1,rm id rakyat merdeka ketua mpr sekaligus wakil...,"{dasar, kemerdekaan, riau, kediaman, menteri, ...",Neutral,0.999451
1,61859,"Di Depan BEM PTNU Se-Nusantara, Bahlil Dorong ...",RM.id Rakyat Merdeka - Menteri Investasi/Kepa...,"Sabtu, 1 Juni 2024 15:46 WIB",,https://rm.id/,https://rm.id/baca-berita/ekonomi-bisnis/22264...,2024-06-01 16:33:36.257516,17,False,DWI ILHAMIEditor :,SRI NURGANINGSIH,False,2024-06-01 15:46:00,1,rm id rakyat merdeka menteri investasi kepala ...,"{juta, perguruan, kepala, digit, sektor, kesei...",Neutral,0.713130
2,61858,Jokowi Minta Pertamina Dan PLN Geber Energi Hi...,RM.id Rakyat Merdeka - Presiden Jokowi mengim...,"Sabtu, 1 Juni 2024 15:43 WIB",,https://rm.id/,https://rm.id/baca-berita/ekonomi-bisnis/22264...,2024-06-01 16:33:36.249971,17,False,FIRSTY HESTYARINI,FIRSTY HESTYARINI,False,2024-06-01 15:43:00,1,rm id rakyat merdeka presiden jokowi mengimbau...,"{mengimbau, riau, gelar, ekspor, panas, energi...",Positive,0.969753
3,61857,Update Haji: 146 Ribu Jemaah Telah Tiba di Tan...,RM.id Rakyat Merdeka - Berdasarkan laporan Pe...,"Sabtu, 1 Juni 2024 15:05 WIB",,https://rm.id/,https://rm.id/baca-berita/humaniora/222640/upd...,2024-06-01 16:33:36.243274,17,False,UJANG SUNDA,UJANG SUNDA,False,2024-06-01 15:05:00,1,rm id rakyat merdeka berdasarkan laporan petug...,"{jemaah, arab, orang, indonesia, wafatsementar...",Neutral,0.999562
4,61856,"Perolehan Suara Melorot, Ketua PPP DKI Didesak...",RM.id Rakyat Merdeka - Kinerja pengurus Parta...,"Sabtu, 1 Juni 2024 15:04 WIB",,https://rm.id/,https://rm.id/baca-berita/parpol/222641/perole...,2024-06-01 16:33:36.236495,17,False,DEDE ISWADI IDRISEditor :,FAQIH MUBAROK,False,2024-06-01 15:04:00,1,rm id rakyat merdeka kinerja pengurus partai p...,"{fakta, meraih, terbukti, wilayah, zaky, mente...",Neutral,0.978405


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
